## Import dependencies

In [ ]:
import os
import pandas as pd
import json
import importlib
import notebook_tools as tools
import pygwalker as pyg
from pandasql import PandaSQL
pdsql = PandaSQL()
tools.configure()

# Configure AWS Profile

Note that the value of `AWS_PROFILE` should match a valid profile in `~/.aws/config` on your host machine (which we mount to this container in the Docker Compose template).

If `sts get-caller-identity` fails, it typically means that your AWS CLI profile on your host is not configured, credentials have expired (e.g. if using SSO). The first step to troubleshooting would be to try using `aws sts get-caller-identity --profile <profile_you_use_below>` from a shell directly on the host. If that fails, the problem is likely not related to this project or container configuration.

In [ ]:
# CHANGE TO WHATEVER PROFILE YOU WANT TO USE LOCALLY
os.environ["AWS_PROFILE"] = "personal-mgmt-admin"
!aws sts get-caller-identity

# Download CUR from AWS S3

The `aws s3 sync` CLI command will only download new or changed file that match the query filters.

In [ ]:
BUCKET_NAME = "temp-cost-reports-123456789012"
BUCKET_REGION = 'us-west-2'
OBJECT_PREFIX=""

!aws s3 ls "s3://{BUCKET_NAME}/{OBJECT_PREFIX}" --recursive | grep  ".parquet$"
!aws s3 sync "s3://{BUCKET_NAME}/{OBJECT_PREFIX}" "s3" --exclude "*" --include "*.parquet"

# Load AWS CUR from Parquet files

Limiting columns can reduce memory footprint and speed things up.

In [ ]:
columns_to_import=[
    'line_item_line_item_type', 
    'line_item_usage_start_date', 
    'line_item_usage_end_date', 
    'line_item_usage_account_id', 
    'line_item_resource_id', 
    'product_region', 
    'line_item_usage_type',
    'line_item_line_item_description', 
    'line_item_usage_amount', 
    'line_item_operation',
    'pricing_unit',
    'line_item_unblended_rate', 
    'product_product_family',
    'product_product_name', 
    'line_item_product_code',
    'product_servicecode', 
    'product_servicename', 
    'product_group', 
    'product_group_description', 
    'product_storage_family', 
    'product_compute_family',
    'product_instance_type_family',
    'product_instance_family', 
    'product_cpu_architecture', 
    'line_item_unblended_cost', 
    'product_region_code',
    'product_location_type', 
    'product_location',
    'product_from_region_code', 
    'product_from_location', 
    'product_from_location_type', 
    'line_item_availability_zone', 
    'product_transfer_type'
]

df = tools.import_cur_to_df(
    path_pattern="./s3/**/*.parquet",
    exclude_row_filters=[
        [("line_item_unblended_cost", ">=", 0.01)],
    ],
    columns_to_import=columns_to_import,
    new_column_names_file="./config/column_renames.json",
    parse_resource_ids=True
)

# Overview of data

## Column description

In [ ]:
tools.describe_df(df)

## Sample rows from source file

In [ ]:
display(df.sample(n=2))

# Example - running SQL directly on dataframe

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
query = """
SELECT 
    resource_type, 
    sum(quantity) as usage, 
    pricing_unit, 
    sum(cost) as cost
FROM 
    df
GROUP BY 
    resource_type,
    pricing_unit
ORDER BY cost ASC
"""

result = pdsql(query)
display(result.sample(5))

# Example - Creating visualizations

The "spec" value is a local file where you can save the visualization configuration so that it's loaded on subsequent runs of this notebook.

In [ ]:
walker = pyg.walk(
    df,                # your dataset, support pandas、polars、modin、spark(not recommended)
    spec="./pywalker_specs/example.json",       # this json will save your chart state, you need to click save button in ui mannual when you finish a chart, 'autosave' will be supported in the future.
    use_preview=True,        # set `use_preview=True`, pygwalker will render preview charts when kernel stop.
)